# Agent Guided Conversations

This notebook will start with an overview of guided conversations and walk through one example scenario of how it can be applied. Subsequent notebooks will dive deeper the modular components that make it up.

## Motivating Example - Education

We focus on an elementary education scenario. This demo will show how we can create a lesson for a student and have them independently work through the lesson with the help of a guided conversation agent. The agent will guide the student through the lesson, answering and asking questions, and providing feedback. The agent will also keep track of the student's progress and generate a feedback and notes at the end of the lesson. We highlight how the agent is able to follow a conversation flow, whilst still being able to exercise judgement to answer and keeping the conversation on track over multiple turns. Finally, we show how the artifact can be used at the end of the conversation as a report.

## Guided Conversation Input

### Artifact
The artifact is a form, or a type of working memory for the agent. We implement it using a Pydantic BaseModel. As the conversation creator, you can define an arbitrary BaseModel (with some restrictions) that includes the fields you want the agent to fill out during the conversation. 

### Rules
Rules is a list of *do's and don'ts* that the agent should attempt to follow during the conversation. 

### Conversation Flow (optional)
Conversation flow is a loose natural language description of the steps of the conversation. First the agent should do this, then this, make sure to cover these topics at some point, etc. 
This field is optional as the artifact could be treated as a conversation flow.
Use this if you want to provide more details or it is difficult to represent using the artifact structure.

### Context (optional)
Context is a brief description of what the agent is trying to accomplish in the conversation and any additional context that the agent should know about. 
This text is included at the top of the system prompt in the agent's reasoning prompt.

### Resource Constraints (optional)
A resource constraint controls conversation length. It consists of two key elements:
- **Unit** defines the measurement of length. We have implemented seconds, minutes, and turns. An extension could be around cost, such as tokens generated.
- **Mode** determines how the constraint is applied. Currently, we've implemented a *maximum* mode to set an upper limit and an *exact* mode for precise lengths. Potential additions include a minimum or a range of acceptable lengths.

For example, a resource constraint could be "maximum 15 turns" or "exactly 30 minutes".

In [1]:
from pydantic import BaseModel, Field

from guided_conversation.guided_conversation_agent import GCInput
from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you. \
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)


# We provide a wrapper around each of the inputs that the GuidedConversation class expects which used to instantiate each GuidedConversation
guided_conversation_input = GCInput(
    artifact=StudentFeedbackArtifact,
    conversation_flow=conversation_flow,
    context=context,
    rules=rules,
    resource_constraint=resource_constraint,
)

### Kickstarting the Conversation

Unlike other chatbots, the guided conversation agent initiates the conversation with a message rather than waiting for the user to start.

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from guided_conversation.guided_conversation_agent import GuidedConversation

# Initialize the agent
kernel = Kernel()
service_id = "gc_main"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name="gpt-4o-2024-05-13",
    api_version="2024-05-01-preview",
    ad_token_provider=token_provider,
)
kernel.add_service(chat_service)
guided_conversation_agent = GuidedConversation(gc_input=guided_conversation_input, kernel=kernel, service_id=service_id)

# Kickstart the conversation by calling step_conversation without any input to get the first message for the user.
response = await guided_conversation_agent.step_conversation()

# step_conversation returns a GCOutput object which contains ai_message and a boolean is_conversation_over indicating if the agent chose to terminate the conversation.
# This object could be extended to include more information if desired.
print(response.ai_message)


Hi David! We're going to write a fun acrostic poem today. An acrostic poem is a type of poem where the first letters of each line spell out a word or phrase. Let me show you how to write one. First, choose a word or phrase you like. Then, you write that word or phrase vertically down the page. Next, think of a word or phrase for each letter. Finally, write these words or phrases next to the corresponding letters to create your poem. Ready?


In [3]:
# Any helper functions go here.

from guided_conversation.utils.conversation_helpers import ConversationMessageType


def get_last_reasoning_message(guided_conversation: GuidedConversation) -> str:
    """Given a instance of the GuidedConversation class, this function returns the last reasoning message in the conversation if it exists."""
    messages = guided_conversation.chat_history.conversation_messages
    msg = "No previous reasoning message found."
    for message in reversed(messages):
        if message.type == ConversationMessageType.REASONING:
            msg = message.content
            break
    return msg

Let's now reply as the student to the agent's message and see what happens. This is the typical flow of a guided conversation. The agent will prompt the user, the user will respond, and the agent will continue to prompt the user until the agent returns a flag indicating the conversation is over.

In [4]:
user_input = "Ok it's almost summer, I'll try to write a poem about that."

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)

print(response.ai_message)

Great choice, David! To help you get started, here's an example of an acrostic poem using the word HAPPY:
```
Having fun with friends all day,
Awesome games that we all play.
Pizza parties on the weekend,
Puppies we bend down to tend,
Yelling yay when we win the game
```
Now it's your turn. Go ahead and write your acrostic poem about 'summer'. Remember to write each letter of 'summer' vertically and then add a phrase or word for each letter. Let your creativity shine!


### The Agenda
Usually after the first message from the user, the agent will generate an initial agenda for the conversation. 
Let's examine what it currently looks like. Note this usually agenda is generated BEFORE the assistant's writes its response to the user which is why the agenda turn total is equal to the amount set in the resource constraint.

In [5]:
# Get string representation of the agenda object. We use this same function for formatting into reasoning prompts.
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())

Current agenda:
1. [1 turn] Provide example of an acrostic poem
2. [3 turns] Have David write his own acrostic poem
3. [2 turns] Review poem and provide initial feedback
4. [2 turns] Prompt David to revise poem and provide final feedback
5. [1 turn] Additional encouragement and exploration (if needed)
Total = 9 turns


Now let's give the agent and incomplete poem.

In [6]:
user_input = """Here is my poem so far.
Sun shines alot
U is for ukulele
My friends visit to play basketball
M
E
R"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You're off to a great start, David! You have a good beginning with 'Sun shines a lot' and 'U is for ukulele,' and 'My friends visit to play basketball.' Now, let's complete the poem. Can you think of a phrase or word for the letters 'M', 'E', and 'R'? For example, for 'M', you could write 'Many beach trips.' Give it a try and let's see what you come up with!


The agent tries to guide us to keep writing the poem! 

Let's try to push our luck and have the agent write the rest for us. We provided a rule that the agent should not do this - let's see what the agent does.

In [7]:
user_input = """I got pretty far can you write the rest for me?"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

I know you can finish it, David! You've done an awesome job so far. Let's complete it together. For the letter 'M', you could write 'Many beach trips.' For 'E', how about 'Eating ice cream'? And for 'R', maybe 'Running around the park.' Try to come up with your own ideas or use these suggestions. You can do it!


Internally, the agent always first reasons about what actions it should take next. Let's see what the agent's reasoning was for this turn. This can often help us understand where the agent went wrong.

After we will continue the conversation for a few turns, with the agent guiding us to complete the poem.

In [8]:
# Get the last reasoning message.
print(get_last_reasoning_message(guided_conversation_agent))

### Step-by-Step Reasoning

David has asked if I can write the rest of the poem for him, which is against the rules. Instead of writing it for him, I'll gently encourage him to continue by himself and provide some support and ideas to help him complete the poem.

### Actions

1. **Encourage David to Complete the Poem**:
   - Encourage him by explaining that he has already done a great job and that he can finish it himself.
   - Provide suggestions for the remaining letters 'M', 'E', and 'R' as ideas to inspire him.

### Updated Agenda

1. **Encourage and guide David to complete the acrostic poem** (2 turns, cumulative total = 5 turns)
2. **Review poem and provide initial feedback** (2 turns, cumulative total = 7)
3. **Prompt David to revise poem and provide final feedback** (2 turns, cumulative total = 9)
4. **Final exploration and summary** (1 turn, cumulative total = 10)

### Final Recommendation

- **Update Agenda**:
  - **Items**:
    1. Encourage and guide David to complete the ac

In [9]:
user_input = "What other things start with e that I could write about?"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Sure, here are some ideas that start with 'E': 
- Eating ice cream
- Exploring new places
- Enjoying the sunshine
- Exciting adventures

Pick one that you like or feel free to come up with your own! Let your creativity guide you.


In [10]:
user_input = """Sun shines alot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Great job, David! Here is the poem you wrote:
```
Sun shines a lot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach
```
You've included some wonderful activities and summer fun! Here are a few things you did really well:
- You used a variety of activities that start with the letters of "summer."
- You conveyed the theme of summer effectively.
One thing you might consider improving:
- Try to vary the beginning of each line a bit more instead of repeating "My friends" to keep the poem fresh and interesting.
Would you like to make any changes? If so, go ahead and revise or let me know if you're happy with how it is!


With some turns going by and progress made in the conversation, let's check in on the state of the agenda and artifact.

If the agent has chosen to update the agenda, we will see the updated agenda. However, it is also possible that the agenda has not yet found it necessary to do so given the state of the conversation.

We should see that the agent has updated the artifact with the current state of the poem since the student has provided it in the previous message.

In [11]:
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current agenda:
1. [2 turns] Review poem and provide initial feedback
2. [2 turns] Prompt David to revise poem and provide final feedback
3. [1 turn] Final exploration and summary
Total = 5 turns
Current artifact:
{'student_poem': 'Sun shines a lot\nU is for ukulele\nMy friends visit to play basketball\nMy friends also visit to play soccer\nEating lots of popsicles\nRoad trips to the beach', 'initial_feedback': 'Unanswered', 'final_feedback': 'Unanswered', 'inappropriate_behavior': ['Student asked the assistant to write the poem for them.']}


In [12]:
user_input = """Here are my updates
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach

But I don't really know what to do for the two my"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You've done a fantastic job with the updates, David! For the lines starting with 'M,' how about changing one to something like 'Making sandcastles on the beach' or 'Meeting new friends at the pool'? Here’s how your poem looks now:
```
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach
```
Just change one of the 'My friends' lines to add more variety. Give it a try and let me know what you come up with!


In [13]:
user_input = """Ok here is my revised poem

Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Amazing work, David! Your final poem beautifully captures the essence of summer:
```
Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach
```
You did a fantastic job varying the lines and adding descriptive language. The images you created with phrases like 'Moving butterflies everywhere' and 'Eating lots of chilly popsicles' are very vivid. Well done!


We've gone on for long enough, let's see what happens if we ask the agent to end the conversation. 

And finally we will print the final state of the artifact after the final update.

In [14]:
user_input = "I'm done for today, goodbye!!"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

No artifact change during final update due to: No tool was called


I will terminate this conversation now. Thank you for your time!


In [15]:
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current artifact:
{'student_poem': 'Sun warms the super fun days!\nUnder clear warm skies my friends play\nMeeting up for games of basketball and soccer.\nMoving butterflies everywhere\nEating lots of chilly popsicles in the sun\nRoad trips to the hot beach', 'initial_feedback': "Great job, David! You've included some wonderful activities and summer fun! Here are a few things you did really well:\n - You used a variety of activities that start with the letters of 'summer.'\n - You conveyed the theme of summer effectively.\nOne thing you might consider improving:\n - Try to vary the beginning of each line a bit more instead of repeating 'My friends' to keep the poem fresh and interesting.", 'final_feedback': 'David did a wonderful job revising his poem. He varied the beginning of each line and added more descriptive words to enhance the imagery in his acrostic poem. His creativity shines through in the final version.', 'inappropriate_behavior': ['Student asked the assistant to write the